# 第四章 在 LangChain 中调用 OpenAI Function Calling

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、Pydantic语法](#二、Pydantic语法)
     - [2.1 简单创建Python类](#2.1-简单创建Python类)
     - [2.2 使用Pydantic创建类](#2.2-使用Pydantic创建类)
 - [三、 基于Pydantic的OpenAI函数定义](#三、基于Pydantic的OpenAI函数定义)
     - [3.1 构造OpenAI的function](#3.1-构造OpenAI的function)
     - [3.2 通过langchain使用function](#3.2-通过langchain使用function)
     - [3.3 强制使用function](#3.3-强制使用function)
 - [四、使用function](#四、使用function)
     - [4.1 链式使用](#3.1-链式使用)
     - [4.2 使用多个function](#4.2-使用多个function)

# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

# 二、Pydantic语法

Pydantic 是 python 的“数据验证库”。
- 与 python 类型注释一起工作。但是，与静态类型检查不同，它们在运行时被积极地用于数据验证和转换。
- 提供内置方法来序列化/反序列化模型到 JSON ，字典等。
- LangChain 利用 Pydantic 创建 JsONScheme 描述函数。

## 2.1 简单创建Python类

在标准python中，可以创建一个`User`类，拥有`name`、`age`、`email`三种属性值。直接进行创建，不能对字段类型进行约束，即年龄中可能传入不合规的字符串类型。

In [1]:
from typing import List
from pydantic import BaseModel, Field

In [2]:
# 创建User类
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [3]:
# 生成user对象
foo = User(name="Joe",age=32, email="joe@gmail.com")

# 输出foo中 name字段
print(foo.name)

Joe


In [4]:
# 生成user对象
foo = User(name="Joe",age="bar", email="joe@gmail.com")

# 输出foo中 name字段
# name字段填写的是字符串类型，但能够创建成功并输出
print(foo.age)

bar


## 2.2 使用 Pydantic 创建类

使用 Pydantic 创建类，可以对类的属性值进行格式约束。在创建类的时候会进行格式验证，如果格式不符合要求会报错。

In [5]:
# 使用 Pydantic创建pUser类别，说明age使用int类型
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [6]:
# 生成user对象
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

# 输出foo中 name字段
print(foo_p.name)

Jane


In [7]:
# 使用了pydantic，由于age不是int，因此会报错。输出报错内容
try:
    foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
except Exception as e:
    print(e)

1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


In [8]:
class Class(BaseModel):
    students: List[pUser]

In [9]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

# 三、基于Pydantic的OpenAI函数定义

## 3.1 构造 OpenAI 的 function

我们创建了一个`WeatherSearch`类，它继承自 Pydantic 的 BaseModel 子类.因此`WeatherSearch`类的所有成员都被具备了数据类型校验机制，该类有一个str类型的成员`airport_code`表示机场代码，并有一个描述信息description，用来说明 airport_code 的作用，在 airport_code 的上方也有一段文本描述信息：输入机场代码，获取该机场的天气信息。这段文本信息是对类`WeatherSearch`的说明，意思是通过机场代码可以查询天气情况。

同时，我们使用 langchain 将这个`WeatherSearch`类转换成openai的函数描述对象：

In [10]:
# 定义WeatherSearch
# WeatherSearch的function需要写注释完成函数的description
class WeatherSearch(BaseModel):
    """输入机场代码，获取该机场的天气信息"""
    airport_code: str = Field(description="输入机场代码查询天气")

In [11]:
# 导入langchain
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [12]:
# 转化为openai function calling模式
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

/Users/jingllinzhou/anaconda3/envs/extractor/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'WeatherSearch',
 'description': '输入机场代码，获取该机场的天气信息',
 'parameters': {'properties': {'airport_code': {'description': '输入机场代码查询天气',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}



接下来，使用 langchain 的`convert_pydantic_to_openai_function`方法将 Pydantic 类转换成了 openai 的函数描述对象。需要的注意的是在定义Pydantic类时文本描述信息不可缺少，如缺少文本描述信息会导致转换时出错。

- `WeatherSearch1`，由于我们没有在`WeatherSearch1`中加入对本身的描述信息，导致在转换时报错。
- `WeatherSearch2`加入对本身的描述信息，因此不会报错。

In [13]:
# WeatherSearch1没有写注释，会报错
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="输入机场代码查询天气")

convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': '输入机场代码查询天气',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [14]:
# 构造WeatherSearch2，不对参数注释
class WeatherSearch2(BaseModel):
    """输入机场代码，获取该机场的天气信息"""
    airport_code: str

In [15]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': '输入机场代码，获取该机场的天气信息',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

## 3.2 通过langchain使用function

为了实现 LLM 对 function 的调用，有一下两种方式

- 在`invoke`中指定 functions
- 执行`invoke`之前使用`bind`方法来绑定函数描述对象

In [22]:
# 导入ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

In [23]:
model = ChatOpenAI(openai_api_key="your_api_key")

In [25]:
# 在invoke中使用openai function功能
model.invoke("今天旧金山的天气怎么样？", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 80, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9e2cd371-b7f5-4abe-970d-92f945fce296-0')

In [26]:
# 直接在构造模型中声明functions，对话时不需要在声明
model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("今天旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 80, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5827a6bd-3777-40ab-ba33-38de4f0ace2c-0')

## 3.3 强制使用function

如果想要强制使用 function，需要在`bind`中增加`function_call`参数。

In [27]:
# 强制使用function，在模型构建时声明function_call
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [28]:
# 基于已经声明的function对话，能够调用function
model_with_forced_function.invoke("旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 88, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aee335d9-35c0-460f-8fde-14d13b8fa989-0')

In [29]:
# 输入hi，强制使用function的模型，即使prompt与函数description无关也会命中
model_with_forced_function.invoke("你好!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"PEK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 78, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-84deaad4-073b-47b7-a0aa-ff409fec5c52-0')

In [30]:
# 输入hi，未强制使用function的模型，prompt与函数description无关不会命中
model_with_function.invoke("你好!")

AIMessage(content='你好!有什么可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 68, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0fbf8b8e-c372-4fbc-af43-83da03c92c17-0')

# 四、使用function

在一般情况下我们会使用 chain 来实现整个问答的流程，接下来我们通过创建 chain 来实现函数调用功能

## 4.1 链式使用

In [31]:
# 引入需要的环境
from langchain.prompts import ChatPromptTemplate

In [32]:
# 使用预定义模版创建聊天代码
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的助手"),
    ("user", "{input}")
])

In [33]:
# 使用prompt + model_with_function 组合
chain = prompt | model_with_function

In [34]:
# 创建命中function的问答
chain.invoke({"input": "今天旧金山的天气怎么样？"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 94, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8b1eba07-1c10-4a50-a6aa-6c3c1c9479ad-0')

In [35]:
# 创建未命中function的问答
chain.invoke({"input": "你好!"})

AIMessage(content='你好! 有什么可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 82, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6882f7fb-94a9-442e-b2a4-7a2bbd168766-0')

## 4.2 使用多个function

我们可以传递一组函数，让 LLM 根据问题上下文决定使用哪个函数。

In [36]:
# 创建 ArtistSearch function
class ArtistSearch(BaseModel):
    """调用此命令可以获得特定艺术家的歌曲名称"""
    artist_name: str = Field(description="要查的艺术家的名字")
    n: int = Field(description="number of results")

In [37]:
# 组装WeatherSearch和ArtistSearch 函数
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [38]:
# 将两个function放入模型
model_with_functions = model.bind(functions=functions)

In [39]:
# 命中 WeatherSearch function
model_with_functions.invoke("旧金山的天气怎么样？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 145, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ca337ef7-9ece-40b3-b85b-8884bda5cd7b-0')

In [40]:
# 命中 ArtistSearch function
model_with_functions.invoke("找到泰勒·斯威夫特的三首歌？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"泰勒·斯威夫特","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 153, 'total_tokens': 185}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6666d6f7-6263-4856-9ad5-679a57ba0c30-0')

In [41]:
# 命中都未命中
model_with_functions.invoke("你好!")

AIMessage(content='你好！有什么可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 135, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-69dbab1f-b9cf-4bba-9076-3efd48984179-0')

# 五、英文版提示

**3.1 构造OpenAI的function**

In [42]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [43]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [44]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

**4.1 链式使用**

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

**4.2 使用多个function**

In [46]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")